In [ ]:
import pandas as pd
import numpy as np
import dask.dataframe as dd

In [ ]:
def work(year):
    df = pd.read_csv(f'{year}.csv')
    df = df[['Country', 'Ft']]
    df['Ft'] = df['Ft'].apply(lambda x : str(x).lower().strip())
    df = df[df['Ft'] != 'nan']
    m = {'AT' : 'Austria', 'BE' : 'Belgium', 'BG' : 'Belgium', 'HR' : 'Croatia', 'CY' : 'Cyprus', 
         'CZ' : 'Czechia', 'DK' : 'Denmark', 'EE' : 'Estonia', 'FI' : 'Finland', 'FR' : 'France', 
         'DE' : 'Germany', 'GR' : 'Greece', 'HU' : 'Hungary', 'IE' : 'Ireland', 'IT' : 'Italy',
         'LV' : 'Latvia', 'LT' : 'Lithuania', 'LU' : 'Luxembourg', 'MT' : 'Malta', 'NL' : 'Netherlands', 
         'PL' : 'Poland', 'PT' : 'Portugal', 'RO' : 'Romania', 'SK' : 'Slovakia', 'SI' : 'Slovenia', 
         'ES' : 'Spain', 'SE' : 'Sweden', 'IS': 'Iceland', 'NO' : 'Norway', 'GB': 'United Kingdom'}

    Ft_replace = {'petrol/electric' : 'petrol-electric', 'hybrid/petrol/e' : 'petrol-electric', 'petrol phev' : 'petrol-electric', 'ng-biomethane':'ng', 'cng':'ng', 'diesel/electric':'diesel-electric','gnl':'ng'}
    countries = list(m.values())
    df['Country'] = df['Country'].replace(m)
    df['Ft'] = df['Ft'].replace(Ft_replace)
    new = df.groupby('Country')['Ft'].value_counts().to_frame().unstack().reset_index()
    new.columns = [x[1] for x in new.columns.to_list()]
    cols = new.columns.to_list()
    cols[0] = 'Country'
    new.columns = cols
    missing = list(set(countries).difference(set(df['Country'])))
    for i in missing:
        new_r = new.iloc[0]

        new_r = new_r.apply(lambda x : np.NaN)

        new_r['Country'] = i

        new_r = pd.DataFrame(new_r).transpose()

        new = pd.concat([new, new_r],ignore_index = True)

    new = new.sort_values('Country')
    tot = pd.DataFrame(new.sum(axis=0)).transpose()
    tot['Country'] = 'EU Total'
    new = pd.concat([new, tot], ignore_index=True)

    if 'other' in new.columns:
        new = new.drop('other', axis=1)

    if 'biodiesel' in new.columns:
        new = new.drop('biodiesel', axis=1)

    if 'unknown' in new.columns:
        new = new.drop('unknown', axis=1)

    if len(new) != 30:
        raise Warning('Not the expected number of rows')
    new.to_csv(f'{year}_new.csv', index = False)